In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from matplotlib.colors import ListedColormap
from matplotlib.colorbar import ColorbarBase
import numpy as np

def LCOH_by_region(file_path, run_name, output_folder,called = False):
    if called == True:
        var_NcapR = pd.read_csv(file_path + 'VAR_NcapR_' + run_name + '.csv', sep = ',')
        var_NcapR = var_NcapR[var_NcapR['2'].str.contains('H2prd')]


        var_NcapR['grp_cnt'] = var_NcapR.groupby('4')['8'].transform('mean')
        var_NcapR = var_NcapR[~var_NcapR['grp_cnt'].duplicated()]

        var_NcapR = var_NcapR.sort_values(by="grp_cnt", ascending=False)

        ### aggreagate regions 

        country_to_region = pd.read_excel('D:/Veda/Veda_models/kinesys_test - Copie/SubRes_Tmpl/SubRES_REZoning_Sol-Win_Trans.xlsx',sheet_name ='AVA')
        country_to_region = country_to_region.iloc[3:].set_axis(country_to_region.iloc[2], axis=1).iloc[:,2:]
        country_to_region['Country'] = ''

        for index, row in country_to_region.iterrows():
            # Split the string in the original column based on the '-' symbol
            parts = row['PSET_PN'].split('-')
            # If there are two parts after splitting, assign the second part to the new column
            country_to_region.at[index, 'Country'] = parts[1].strip()

        # country_to_region.head()

        LCOH_map = country_to_region.merge(var_NcapR, how='left', left_on='Region', right_on='4').reset_index(drop=True)
        LCOH_map = LCOH_map.dropna()

        ## map

        world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

        world_LCOH_map = world.merge(LCOH_map, how='left', left_on='iso_a3', right_on='Country').reset_index(drop=True)
        world_LCOH_map.shape


        colors = ['#d73027', '#fc8d59', '#fee08b', '#d9ef8b', '#91cf60', '#1a9850']

        # Create a colormap from the defined colors
        cmap = ListedColormap(colors[::-1])

        # # Plot the map
        fig, ax = plt.subplots(1, 1, figsize=(15, 8))
        world.plot(ax=ax, color='white', edgecolor='black')
        world_LCOH_map.plot(column='grp_cnt', cmap=cmap, linewidth=0.8, ax=ax, edgecolor='black', legend=False)

        # Add title and customize plot
        plt.title('Levelized cost of H2 by region')
        plt.ylim([-60, 90])
        plt.axis('off')  # Turn off axis

        # Create a color bar
        ax_legend = fig.add_axes([0.4, 0.2, 0.25, 0.02])  # [left, bottom, width, height]
        colorbar = ColorbarBase(ax_legend, cmap=cmap, orientation='horizontal')
        colorbar.set_label('LCOH ($/kgH2)')

        min_val = world_LCOH_map['grp_cnt'].min()
        max_val = world_LCOH_map['grp_cnt'].max()
        num_ticks = len(colors)
        tick_values = np.linspace(min_val, max_val, num_ticks)
        tick_labels = [f'{value:.2f}' for value in tick_values]

        # Set the color bar tick locations and labels
        # colorbar.set_ticks(tick_values)
        colorbar.set_ticklabels(tick_labels)

        colorbar.set_label('LCOH ($/kgH2)')

        plt.savefig(output_folder + 'LCOH_region.pdf'
                    , format ='pdf'
                    ,  bbox_inches='tight')
    else:
        pass